In [2]:
!pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cu

In [3]:
from datasets import load_dataset
dataset = load_dataset("baptle/financial_headlines_market_based")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/420 [00:00<?, ?B/s]

to_hg_data.csv:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8142 [00:00<?, ? examples/s]

In [4]:
dataset['train']['Title'][:10]

['Global Stocks and Commodities Rally on First Trading Day of 2010',
 'Dollar Slumps Amid Worldwide Manufacturing Improvement',
 'Oil Prices Surge Above $81 a Barrel Due to U.S. Cold Weather',
 'S&P 500 Sees 1.6 Percent Increase, Hits 15-Month High',
 "Argentina's Merval Index Reaches Record High, Brazil's Bovespa Surpasses 70,000",
 'Natural Gas and Crude Oil Prices Jump Amid U.S. Cold Snap',
 'Gold and Copper Prices Rise, with Copper Hitting 16-Month High',
 'U.S. Manufacturing Index Reaches Highest Level in Over Three Years',
 ' Exxon Mobil Corp. and Chevron Corp. Lead Gains in U.S. Energy Shares',
 ' Intel Corp. Shares Rally After Upgrade by Robert W. Baird & Co.']

In [5]:
data = dataset['train']['Title'][:1000]

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd

In [7]:
# Инициализируем токенизатор
tokenizer = Tokenizer()

# Обучаем токенизатор на заголовках
tokenizer.fit_on_texts(data)

# Преобразуем заголовки в последовательности чисел
sequences = tokenizer.texts_to_sequences(data)

# Создаем входные и выходные данные
X = []
y = []
for seq in sequences:
    for i in range(1, len(seq)):
        X.append(seq[:i])
        y.append(seq[i])

In [8]:
X[:10], y[:10]

([[71],
  [71, 9],
  [71, 9, 2],
  [71, 9, 2, 55],
  [71, 9, 2, 55, 35],
  [71, 9, 2, 55, 35, 12],
  [71, 9, 2, 55, 35, 12, 117],
  [71, 9, 2, 55, 35, 12, 117, 178],
  [71, 9, 2, 55, 35, 12, 117, 178, 49],
  [71, 9, 2, 55, 35, 12, 117, 178, 49, 38]],
 [9, 2, 55, 35, 12, 117, 178, 49, 38, 248])

x - последовательность цепочек данных контекста, y - соответствующее контексту предсказание. Например, для 71 предсказание 9 для контекста 71, 9 -- 2 и т.д.

$$ h_t = \sigma( Wx_t + Vh_{t-1}  + b) $$


сигмоида необходима для ужимки данных от 0 до 1. b - смещение (задаем ориентир), $$Wx_t$$ - обучаемый параметр, отвечающий за кодирование данных в скритое пространство. $$Vh_{t-1}$$ - обучаемый параметр, апеллирующий прошлым контекстам

In [9]:
# Преобразуем списки в массивы numpy
X = np.asarray(X, dtype="object")
y = np.array(y)

# Дополняем последовательности до одинаковой длины
X = pad_sequences(X)

# Преобразуем y в one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=len(tokenizer.word_index) + 1)

In [10]:
# Создаем модель
model = Sequential() #штука которая помогает подключать слои за слоями и таким образом создавать модели

# Добавляем слой Embedding
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=X.shape[1]))
#первый параметр - длина словаря, второй - итоговый размер вектора, третий - количество слов на входе

# Добавляем слой LSTM
model.add(LSTM(150, return_sequences=False)) #150 - размер "памяти", второй параметр исключает вывод промежуточных результатов

# Добавляем полносвязный слой
model.add(Dense(len(tokenizer.word_index) + 1, activation='softmax'))

# Компилируем модель
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #первый параметр - лосс-функция: категориальная (не бинарная) энтропия
#adam - данные в виде one-hot, третий параметр - градиентный спуск


# Выводим информацию о модели
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Обучаем модель
history = model.fit(X, y, epochs=10, batch_size=1, validation_split=0.2)
#x_train, y_test, epochs - количество подходов, batch_size - проверка сразу каждого случая, validation_split = 0.2 - 80% данных для обучения, 20% - для тестирования

Epoch 1/10
6828/6828 ━━━━━━━━━━━━━━━━━━━━ 65s 9ms/step - accuracy: 0.0516 - loss: 6.4457 - val_accuracy: 0.1160 - val_loss: 5.8032
Epoch 2/10
6828/6828 ━━━━━━━━━━━━━━━━━━━━ 81s 9ms/step - accuracy: 0.1777 - loss: 4.8480 - val_accuracy: 0.1752 - val_loss: 5.5287
Epoch 3/10
6828/6828 ━━━━━━━━━━━━━━━━━━━━ 81s 9ms/step - accuracy: 0.2585 - loss: 3.9129 - val_accuracy: 0.2045 - val_loss: 5.5369
Epoch 4/10
6828/6828 ━━━━━━━━━━━━━━━━━━━━ 80s 9ms/step - accuracy: 0.3486 - loss: 3.1883 - val_accuracy: 0.2127 - val_loss: 5.8469
Epoch 5/10
6828/6828 ━━━━━━━━━━━━━━━━━━━━ 82s 9ms/step - accuracy: 0.4489 - loss: 2.5086 - val_accuracy: 0.2156 - val_loss: 5.9457
Epoch 6/10
6828/6828 ━━━━━━━━━━━━━━━━━━━━ 68s 10ms/step - accuracy: 0.5621 - loss: 1.9164 - val_accuracy: 0.2109 - val_loss: 6.1782
Epoch 7/10
6828/6828 ━━━━━━━━━━━━━━━━━━━━ 75s 9ms/step - accuracy: 0.6548 - loss: 1.5321 - val_accuracy: 0.2209 - val_loss: 6.2723
Epoch 8/10
6828/6828 ━━━━━━━━━━━━━━━━━━━━ 83s 9ms/step - accuracy: 0.7385 - loss: 

в результате модель прекрасно работает на тренировочных данных и совершенно плохо на валидационной выборке

In [12]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (1, 16, 100)                │         153,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (1, 150)                    │         150,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (1, 1534)                   │         231,634 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,606,904 (6.13 MB)

 Trainable params: 535,634 (2.04 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,071,270 (4.09 MB)

In [13]:
history = model.fit(X, y, epochs=10, batch_size=50, validation_split=0.2)

Epoch 1/10
137/137 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.8499 - loss: 0.6103 - val_accuracy: 0.2185 - val_loss: 6.5963
Epoch 2/10
137/137 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - accuracy: 0.8656 - loss: 0.5430 - val_accuracy: 0.2173 - val_loss: 6.5993
Epoch 3/10
137/137 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.8633 - loss: 0.5330 - val_accuracy: 0.2191 - val_loss: 6.6075
Epoch 4/10
137/137 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - accuracy: 0.8655 - loss: 0.5005 - val_accuracy: 0.2144 - val_loss: 6.6179
Epoch 5/10
137/137 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - accuracy: 0.8694 - loss: 0.4899 - val_accuracy: 0.2150 - val_loss: 6.6295
Epoch 6/10
137/137 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.8808 - loss: 0.4407 - val_accuracy: 0.2162 - val_loss: 6.6428
Epoch 7/10
137/137 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.8709 - loss: 0.4634 - val_accuracy: 0.2173 - val_loss: 6.6545
Epoch 8/10
137/137 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.8762 - loss: 0.4562 - val_accu

модель быстрее научилась (метрика accuracy стабильно хороша), но проблемы те же с валидационной выборкой

In [38]:
def generate_text(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0] #слово --> числовая последовательность, возвращает список списков (обращаемся к первому элементу, к сердцевинке)
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre') #заполнение нулем в начале до нужной длины
        predicted = np.argmax(model.predict(token_list), axis=-1) #предсказывание (выбор элемента с наибольшей вероятностью)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word # вынимаем слова из словаря
                break
        seed_text += " " + output_word #добавление слова в последовательность
    return seed_text

# Генерируем новый заголовок
generated_text = generate_text("Dollar", 2, X.shape[1])
print(generated_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Dollar index falls


In [39]:
model.save('news_headline_generator.keras')